In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [ ]:
max_features = 10000 ## vocabulary size
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words= max_features)

print(f"Training data shape{x_train.shape} \n Training Label shape {y_train.shape}")
print(f"Testing data shape{x_test.shape} \n Training Label shape {y_test.shape}")


In [ ]:
sample_data = x_train[0]
sample_label = y_train[0]
print(f"Sample Data : {sample_data} \n Sample Label:{sample_label}")

In [ ]:
# Mapping of Onehot repr back to words
word_index = imdb.get_word_index()
word_index

In [ ]:
word_index_reversed = {v:k for k,v in word_index.items()}
word_index_reversed

In [ ]:
decoded_review = ' '.join(word_index_reversed.get(i-3,'?') for i in sample_data)
decoded_review

In [ ]:
m_len =500
x_train = sequence.pad_sequences(x_train, maxlen=m_len)
x_test = sequence.pad_sequences(x_test, maxlen = m_len)

x_train

In [23]:
##Train Simple RNN
model = Sequential()
model.add(Embedding(max_features,128,input_shape =(m_len,)))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [26]:
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
#create an instance a early stopping callback
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [28]:
history = model.fit(x_train,y_train,
        epochs=10,
        batch_size=32,
        validation_split = 0.2,
        callbacks=[early_stopping])

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 52s 83ms/step - accuracy: 0.5803 - loss: 7423958.0000 - val_accuracy: 0.6170 - val_loss: 0.6324
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 98s 156ms/step - accuracy: 0.6790 - loss: 0.7746 - val_accuracy: 0.6682 - val_loss: 0.5856
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 152s 172ms/step - accuracy: 0.7613 - loss: 0.6892 - val_accuracy: 0.7204 - val_loss: 0.5471
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 102s 162ms/step - accuracy: 0.8446 - loss: 0.3825 - val_accuracy: 0.7640 - val_loss: 0.4848
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 163ms/step - accuracy: 0.9023 - loss: 0.2691 - val_accuracy: 0.7874 - val_loss: 0.4777
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 136s 154ms/step - accuracy: 0.9279 - loss: 0.1999 - val_accuracy: 0.7658 - val_loss: 0.5750
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 162ms/step - accuracy: 0.9521 - loss: 0.1436 - val_accuracy: 0.7760 - val_loss: 0.5383
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 180s 287ms/step - accuracy: 0.966

In [29]:
model.save('predictor_simple_rnn.h5')